# 🍏 건강 및 피트니스 에이전트와 Bing 그라운딩 🍎

**건강 및 피트니스 에이전트와 Bing 그라운딩** 튜토리얼에 오신 것을 환영합니다! 이 노트북에서는 방법을 보여 드리겠습니다:

1. Azure AI Foundry를 사용하여 프로젝트를 **초기화**합니다.
2. 웹 검색을 위해 **BingGroundingTool**을 사용하여 **에이전트 만들기**.
3. 건강 및 피트니스에 대한 **실제 질문을 합니다**.
4. Bing 쿼리 URL 및 면책 조항을 포함한 **답변을 검색하고 표시**합니다.

### ⚠️ 중요 모델 지원 참고 사항 ⚠️
> Bing 그라운딩은 현재 특정 Azure OpenAI 모델(예: `gpt-4o-0513`)에서만 지원됩니다.
> 
> 지원되는 모델을 지정하고 `"x-ms-enable-preview": "true"` 헤더를 설정합니다.

## 전제 조건
- 에이전트 기본 노트북을 완료하세요. - [1-basics.ipynb](1-basics.ipynb)
- Azure 포털에서 프로비저닝해야 하는 Azure AI Foundry의 Bing 연결로 그라운딩.
자세한 내용은 설명서에서 ["Setup Bing Grounding"](https://learn.microsoft.com/en-us/azure/ai-services/agents/how-to/tools/bing-grounding?tabs=python&pivots=overview#setup) 을 참조하세요.

<img src="./seq-diagrams/bing-api-key.png" width="30%"/>

- 상위 디렉토리에 있는  `.env` 파일에는 다음 변수들이 있습니다:
  ```bash
  PROJECT_CONNECTION_STRING=<your-connection-string>
  MODEL_DEPLOYMENT_NAME=<supported-model>
  BING_CONNECTION_NAME=<the-name-of-your-bing-connection>
  ```

## Bing 그라운딩을 살펴보세요!
웹에서 추가 컨텍스트를 수집할 수 있도록 **Bing으로 그라운딩**한 검색 결과를 에이전트에 통합할 것입니다. 투명성을 위해 Bing 검색 쿼리 링크를 저장하고 표시할 것입니다. 🎉

<br/>

<img src="./seq-diagrams/4-bing-grounding.png" width="30%"/>


## 1. 초기 설정
`.env`에서 환경 변수를 로드하고 에이전트를 관리하기 위해 **AIProjectClient**를 초기화하겠습니다.

In [ ]:
import os
import time
from pathlib import Path

from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import BingGroundingTool, MessageTextContent

# Load environment variables
notebook_path = Path().absolute()
parent_dir = notebook_path.parent
load_dotenv(parent_dir / '.env')

# Initialize AIProjectClient
try:
    project_client = AIProjectClient.from_connection_string(
        credential=DefaultAzureCredential(),
        conn_str=os.environ.get("PROJECT_CONNECTION_STRING")
    )
    print("✅ Successfully initialized AIProjectClient")
except Exception as e:
    print(f"❌ Error initializing project client: {e}")

## 2. Bing 그라운딩 에이전트 생성 🌐
AI Foundry에서 Bing 연결을 가져오고 `BingGroundingTool`을 사용하여 에이전트가 웹을 검색할 수 있도록 합니다. 그런 다음 의학관련 면책 조항이 포함된 새 에이전트를 만듭니다.

`MODEL_DEPLOYMENT_NAME`이 Bing 지원 모델로 설정되어 있는지 확인하고 (예를 들어, `gpt-4o-0513`) 헤더 `{"x-ms-enable-preview": "true"}`를 추가합니다..

In [ ]:
def create_bing_grounded_agent():
    """Create an agent that can use Bing to ground queries with up-to-date info."""
    try:
        # 1. Retrieve Bing connection from your AI Foundry project
        bing_conn_name = os.environ.get("BING_CONNECTION_NAME")
        if not bing_conn_name:
            raise ValueError("BING_CONNECTION_NAME not set in .env")

        bing_connection = project_client.connections.get(connection_name=bing_conn_name)
        conn_id = bing_connection.id
        print(f"Bing Connection ID: {conn_id}")

        # 2. Initialize Bing grounding tool
        bing_tool = BingGroundingTool(connection_id=conn_id)

        # 3. Create an agent that can search with Bing
        agent = project_client.agents.create_agent(
            model=os.environ.get("MODEL_DEPLOYMENT_NAME", "gpt-4o-0513"),
            name="health-bing-agent",
            instructions="""
                You are a health and fitness assistant with Bing search capabilities.
                Always:
                1. Provide disclaimers that you are not a medical professional.
                2. Encourage professional consultation.
                3. Use Bing for real-time references.
                4. Provide brief, helpful answers.
            """,
            tools=bing_tool.definitions,
            # Must pass special preview header to use Bing grounding (subject to change)
            headers={"x-ms-enable-preview": "true"},
        )

        print(f"🎉 Created Bing-grounded agent, ID: {agent.id}")
        return agent
    except Exception as e:
        print(f"❌ Error creating Bing-grounded agent: {e}")
        return None

# Create our Bing-based agent
bing_agent = create_bing_grounded_agent()

## 3. 스레드 시작 및 질문하기 💬
각 사용자 쿼리에 대한 대화 스레드를 만들어 상담원이 Bing으로 검색하여 관련 정보를 찾을 수 있도록 합니다. 다음 단계에서 검토할 수 있도록 모든 `(thread, run)` 쌍을 목록에 저장합니다.

In [ ]:
bing_threads = []

def ask_bing_question(agent, user_query):
    try:
        thread = project_client.agents.create_thread()
        print(f"📝 Created a conversation thread, ID: {thread.id}")

        # Post user query as a message
        user_message = project_client.agents.create_message(
            thread_id=thread.id,
            role="user",
            content=user_query
        )
        print(f"📨 Created user message with query: '{user_query}'")

        # Process the query with the agent
        run = project_client.agents.create_and_process_run(
            thread_id=thread.id,
            assistant_id=agent.id
        )
        print(f"🤖 Run finished with status: {run.status}\n")
        if run.last_error:
            print(f"Error detail: {run.last_error}\n")

        return thread, run
    except Exception as e:
        print(f"❌ Error asking Bing question: {e}")
        return None, None

if bing_agent:
    # We'll ask a few fun questions!
    questions = [
        "What are some new HIIT workout trends I should know about?",
        "What's the current WHO recommendation for sugar intake?",
        "Any news on intermittent fasting for weight management?"
    ]

    for q in questions:
        thr, rn = ask_bing_question(bing_agent, q)
        if thr and rn:
            bing_threads.append((thr, rn))

## 4. Bing 기반 답변 및 쿼리 URL 보기
각 스레드의 메시지를 검색하여 사용자 쿼리와 에이전트의 응답을 모두 인쇄합니다. 또한 실행 단계를 가져와서 **Bing 검색 쿼리 URL**을 표시합니다(데이터의 출처를 표시해야 하는 요구 사항을 준수할 수 있도록). 사용자 친화적인 링크를 형성하기 위해 `api.bing.microsoft.com`을 `www.bing.com`으로 바꿀 수 있습니다.

`RunStep` 개체에는 `.details`가 없으므로 대신 `step[“parameters”]`에서 `'request_url'`을 찾습니다. 발견되면 아마도 Bing 단계일 것입니다.

In [ ]:
def view_bing_conversation(thread_id, run_id):
    try:
        # Print conversation
        messages = project_client.agents.list_messages(thread_id=thread_id)
        print("\n🗣️ Conversation for thread:", thread_id)
        reversed_msgs = list(reversed(messages.data))  # oldest first
        for msg in reversed_msgs:
            role = msg.role.upper()
            if msg.content:
                for c in msg.content:
                    if hasattr(c, 'text') and c.text:
                        print(f"{role}: {c.text.value}\n")

        # Retrieve run steps to get Bing search query link
        run_steps = project_client.agents.list_run_steps(thread_id=thread_id, run_id=run_id)
        steps_data = run_steps.get('data', [])
        if steps_data:
            print("\n🔎 Bing run steps:")
            for step in steps_data:
                # 'parameters' is typically where BingGroundingTool stores request_url
                params = step.get('parameters', {})
                if 'request_url' in params:
                    original_url = params['request_url']
                    friendly_url = original_url.replace("api.bing.microsoft.com", "www.bing.com")
                    print(f"    Bing search URL: {friendly_url}")
        else:
            print("No run step data found for this conversation.")
    except Exception as e:
        print(f"❌ Error viewing Bing conversation: {e}")

# Display all queries and agent responses
if bing_threads:
    for (thr, rn) in bing_threads:
        view_bing_conversation(thr.id, rn.id)

## 5. 정리 및 모범 사례
작업을 완료한 후에는 선택적으로 에이전트를 삭제할 수 있습니다. 프로덕션 환경에서는 반복 사용을 위해 보관할 수 있습니다.

### 모법 사례
1. **정확성** - Bing 검색 결과에 고지 사항 또는 일부 정보가 포함될 수 있습니다. 신뢰할 수 있는 출처를 통해 확인하도록 권장하세요.
2. *Bing 검색어 표시** - Bing의 사용 및 표시 요건을 준수하기 위해 상담원의 응답에 **website URLs**과 **Bing Query Display**을 모두 표시합니다(위에 표시됨). 모델에 인용이 포함되어 있으면 인용도 표시하세요.
3. **한도** - Bing의 사용량, 요금 제한 또는 정책 제약 조건에 대해 주의합니다.
4. **개인정보** - 민감한 데이터가 전송되지 않도록 검색 쿼리를 필터링합니다.
5. **평가** - 반복적인 개선을 위해 `azure-ai-evaluation`을 사용합니다.

In [ ]:
def cleanup_bing_agent(agent):
    if agent:
        try:
            project_client.agents.delete_agent(agent.id)
            print(f"🗑️ Deleted Bing-grounded agent: {agent.name}")
        except Exception as e:
            print(f"❌ Error cleaning up agent: {e}")
    else:
        print("No agent to clean up.")

# Uncomment if you want to remove the agent now
cleanup_bing_agent(bing_agent)

# 축하합니다! 🎉
다음과 같은 **Bing 기반 건강 및 피트니스 에이전트**를 구축했습니다:
1. Bing으로 웹을 **검색**합니다.
2. 고지 사항과 함께 건강/피트니스 관련 질문에 **답변**.
3. 참조 및 Bing 검색어 링크를 **포함**합니다.

강력한 어드바이저를 구축하기 위해 BingGroundingTool을 다른 도구(예: **FileSearchTool**, **CodeInterpreterTool**)와 결합하여 이 접근 방식을 자유롭게 확장할 수 있습니다.

#### [5-agents-aisearch.ipynb](./5-agents-aisearch.ipynb)로 이동합니다.